# 1. Aggregate results for analysis

Read the complete metadataset and generate a clean version for analysis.


## Instructions:
1. make sure that the latest metadataset is present in the `TabSurvey` folder (two levels up from this script). The latest metadataset can be downloaded from the GCP bucket here: `tabzilla-results/metadatasets`.

2. Run the first two cells in thie notebook, to create a metadataset `metadataset_df`. This dataframe includes all results from our experiments on datasets with clasisfication and binary targets. The remaining cells are divided into two sections: (A) and (B).

3. Modify and run the cells in section (A) to create a dataframe `analysis_df`, which is a subset of `metadataset_df`. This subset should include all results you want to include in later analyses. Section (A) in this notebook selects results such that each algorithm has results for at least 85% of all datasets.

4. After `analysis_df` is defined in section (A), run cells in section (B) to "tune" all algorithms on each dataset, and calculate normalized results and rankings. These cells create four "cleaned" results files, which will be used for all subsequent analysis: 
- `./cleaned_results/tuned_aggregated_results.csv`: performance of each tuned algorithm on each dataset, where performance is averaged over all 10 folds. 
- `./cleaned_results/tuned_fold_results.csv`: performance of each tuned algorithm on each dataset fold.
- `./cleaned_results/tuned_aggregated_results_with_default.csv`: same as `tuned_aggregated_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm
- `./cleaned_results/tuned_fold_results_with_default.csv`: same as `tuned_fold_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm

In [1]:
from pathlib import Path
import pandas as pd
from analysis_utils import get_tuned_alg_perf

metadata_folder = Path("../../TabSurvey")

metadataset_df = pd.read_csv(metadata_folder / "metadataset.csv")
errors_df = pd.read_csv(metadata_folder / "metadataset_errors.csv")

# keep only binary and classification datasets. we have some results for regression datasets, which are not used.
metadataset_df = metadataset_df.loc[metadataset_df["target_type"].isin(["binary", "classification"]), :]

# make sure that the cleaned_results folder exists
output_folder = Path("./cleaned_results")
output_folder.mkdir(exist_ok=True)    

### Print number of results per dataset and alg

In [2]:
# for each alg, for how many datasets are there results?
print(f"for each alg: number of datasets with results (out of {len(metadataset_df['dataset_name'].unique())})")
print(metadataset_df.groupby("alg_name")["dataset_name"].apply(lambda x: len(set(x))).sort_values())

print(f"for each dataset: number of algs with results (out of {len(metadataset_df['alg_name'].unique())})")
metadataset_df.groupby("dataset_name")["alg_name"].apply(lambda x: len(set(x))).sort_values()

for each alg: number of datasets with results (out of 176)
alg_name
TabPFNModel            63
NAM                    80
DeepFM                 90
TabTransformer        124
SAINT                 138
NODE                  141
SVM                   143
DANet                 147
rtdl_FTTransformer    148
VIME                  163
STG                   164
CatBoost              165
LightGBM              165
KNN                   167
LinearModel           168
TabNet                168
RandomForest          173
XGBoost               174
rtdl_ResNet           174
MLP                   175
DecisionTree          175
rtdl_MLP              176
Name: dataset_name, dtype: int64
for each dataset: number of algs with results (out of 22)


dataset_name
openml__poker-hand__9890             5
openml__covertype__7593              7
openml__Devnagari-Script__167121     8
openml__albert__189356               9
openml__CIFAR_10__167124            10
                                    ..
openml__profb__3561                 22
openml__dresses-sales__125920       22
openml__pc1__3918                   22
openml__heart-h__50                 22
openml__kc2__3913                   22
Name: alg_name, Length: 176, dtype: int64

# A. Dataset inclusion/exclusion

**In this notebook: selected-18-algs:**
* We use a list of 18 algs (excluding 3 that had lots of errors.)
* We take only the datasets where each of these algs produce a result. This is ~100 datasets.

In [3]:
print(f"for each dataset: number of algs with results")
alg_counts = metadataset_df.groupby("dataset_name")["alg_name"].agg(lambda x: len(set(x))).sort_values()
print(alg_counts.head())

for each dataset: number of algs with results
dataset_name
openml__poker-hand__9890             5
openml__covertype__7593              7
openml__Devnagari-Script__167121     8
openml__albert__189356               9
openml__CIFAR_10__167124            10
Name: alg_name, dtype: int64


In [7]:
# drop:
# - TabPFN (only runs for small datasets)
# - NAM (lots of errors, long runtime)
# - DeepFM (not implemented for multi-class)

selected_algs = [
    "SAINT",
    "NODE",
    "SVM",
    "DANet",
    "rtdl_FTTransformer",
    "VIME",
    "STG",
    "CatBoost",
    "LightGBM",
    "KNN",
    "LinearModel",
    "TabNet",
    "RandomForest",
    "XGBoost",
    "rtdl_ResNet",
    "MLP",
    "DecisionTree",
    "rtdl_MLP",
    # "NAM",
    # "TabPFNModel",
    # "DeepFM",
]

test_df = metadataset_df.loc[metadataset_df["alg_name"].isin(selected_algs), :]

# keep only datasets where all selected algs produce a result
alg_count = test_df.groupby("dataset_name")["alg_name"].apply(lambda x: len(set(x)))

keep_datasets = alg_count[alg_count == len(selected_algs)].index

print(f"keeping {len(keep_datasets)} datasets")

keep_df = test_df.loc[test_df["dataset_name"].isin(keep_datasets), :]


keeping 104 datasets


### Slightly more data prep

Note: We will keep all algs, regardless of how many datasets they have results for.

In [8]:
analysis_df = keep_df

print("after removing datasets: number of datasets with results")
dataset_counts = analysis_df.groupby("alg_name")["dataset_name"].agg(lambda x: len(set(x))).sort_values()
print(dataset_counts)

after removing datasets: number of datasets with results
alg_name
CatBoost              104
rtdl_FTTransformer    104
XGBoost               104
VIME                  104
TabNet                104
SVM                   104
STG                   104
SAINT                 104
RandomForest          104
NODE                  104
MLP                   104
LinearModel           104
LightGBM              104
KNN                   104
DecisionTree          104
DANet                 104
rtdl_MLP              104
rtdl_ResNet           104
Name: dataset_name, dtype: int64


# B. Tune and rank algorithms for each dataset

**Note**: At this point, you should have a dataframe called `analysis_df`, which contains all results you want to include in the remainder of the analysis. 

The code below performs hyperparameter tuning & ranking of each alg, and writes four cleaned results files to the directory `./cleaned_results`. 

In [9]:
metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
]

obj_type_list = [
    "maximize",
    "maximize",
    "minimize",
]
result_df_dict = {}

### Bookkeeping

In [10]:
# replace alg name with display name
from analysis_utils import ALG_DISPLAY_NAMES, ALG_TYPES
analysis_df.loc[:, "alg_name"] = analysis_df["alg_name"].apply(lambda x: ALG_DISPLAY_NAMES[x])

# add alg type
analysis_df.loc[:, "alg_type"] = analysis_df["alg_name"].apply(lambda x: ALG_TYPES[x])

/tmp/ipykernel_32643/1589380835.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_df.loc[:, "alg_name"] = analysis_df["alg_name"].apply(lambda x: ALG_DISPLAY_NAMES[x])
/tmp/ipykernel_32643/1589380835.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_df.loc[:, "alg_type"] = analysis_df["alg_name"].apply(lambda x: ALG_TYPES[x])


In [11]:
# add a copy of each "default" hparam row, to treat this as a separate alg
default_rows = analysis_df.loc[analysis_df["hparam_source"] == "default"].copy()
default_rows.loc[:, "alg_name"] = default_rows["alg_name"].apply(lambda x: x + " (default)")

# remove TabPFN and LinearModel, since these only have one hparam set
default_rows = default_rows.loc[~(default_rows["alg_name"].str.contains("TabPFNModel") | default_rows["alg_name"].str.contains("LinearModel")), :]

# append these to the metadataset
analysis_df_with_default = pd.concat([analysis_df, default_rows], ignore_index=True)

### produce cleaned results files

first, each algorithm is tuned for each dataset fold (10 folds per dataset), using all available hyperparameter samples. we then calculate the normalized and ranked performance for each algorithm over all datasets.

the following loop produces four files:
- `./cleaned_results/tuned_aggregated_results.csv`: performance of each tuned algorithm on each dataset, where performance is averaged over all 10 folds. 
- `./cleaned_results/tuned_fold_results.csv`: performance of each tuned algorithm on each dataset fold.
- `./cleaned_results/tuned_aggregated_results_with_default.csv`: same as `tuned_aggregated_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm
- `./cleaned_results/tuned_fold_results_with_default.csv`: same as `tuned_fold_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm

In [12]:
tuned_result_dfs = {}
for drop_default in [True, False]:
    for i, (metric, objective_type) in enumerate(zip(metric_list, obj_type_list)):

        test_metric_col = metric + "__test"

        if drop_default:
            df = analysis_df.copy()
        else:
            df = analysis_df_with_default.copy()

        tuned_alg_perf = get_tuned_alg_perf(df, metric=metric)

        # NOTE: this "tunes" each algorithm for each training fold separately. so each of the 10 folds might use different hparams.
        tuned_result_dfs[metric] = tuned_alg_perf

        ##############################
        ### STEP 1: TREAT EACH FOLD AS SEPARATE DATASET

        result_col = test_metric_col
        
        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = tuned_alg_perf.groupby("dataset_fold_id").agg({result_col: ["min", "max"]}).reset_index()

        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        tuned_alg_perf = tuned_alg_perf.merge(overall_bounds, on="dataset_fold_id", how="left")

        # add normalized metric
        tuned_alg_perf.loc[:, "normalized_" + result_col] = (tuned_alg_perf[result_col] - tuned_alg_perf[result_col + "_min"]) / (tuned_alg_perf[result_col + "_max"] - tuned_alg_perf[result_col + "_min"])

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        tuned_alg_perf.loc[:, f"{metric}_rank"] = tuned_alg_perf.groupby(["dataset_fold_id"])[result_col].rank(method="min", ascending=ascending).values

        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_fold_id", 
            "normalized_" + result_col,
            f"{metric}_rank",
            result_col + "_min",
            result_col + "_max"
        ]

        if i == 0:
            fold_tuned_df = tuned_alg_perf.copy()
        else:
            fold_tuned_df = fold_tuned_df.merge(tuned_alg_perf[merge_cols], on=["alg_name", "dataset_fold_id"])

        ##############################
        ### STEP 2: AVERAGE OVER FOLDS

        if i == 0:
            agg_dict = {
                test_metric_col: ["median", "mean"],
                "time__train": ["median", "mean"],
                # "dataset_name": ["count"],
            }
        else:
            agg_dict = {
                test_metric_col: ["median", "mean"],
            }

        # aggregate over folds: take the mean & median performance over each fold
        agg_tuned_alg_perf = tuned_alg_perf.groupby(["alg_name", "dataset_name"]).agg(agg_dict).reset_index()

        # rename the multiindex cols
        new_cols = []
        for c in agg_tuned_alg_perf.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        agg_tuned_alg_perf.columns = new_cols


        # define the target metric column, we will use this value for all plots
        result_col = test_metric_col + "_mean"

        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = agg_tuned_alg_perf.groupby("dataset_name").agg({result_col: ["min", "max"]}).reset_index()
        
        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        
        agg_tuned_alg_perf = agg_tuned_alg_perf.merge(overall_bounds, on="dataset_name", how="left")

        # add normalized metric
        agg_tuned_alg_perf.loc[:, "normalized_" + result_col] = (agg_tuned_alg_perf[result_col] - agg_tuned_alg_perf[result_col + "_min"]) / (agg_tuned_alg_perf[result_col + "_max"] - agg_tuned_alg_perf[result_col + "_min"])

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        # rank according to mean performance over all folds
        agg_method = "mean"

        # rank everything
        agg_tuned_alg_perf.loc[:, f"{metric}_rank_{agg_method}"]  = \
            agg_tuned_alg_perf.groupby(["dataset_name"])[test_metric_col + "_" + agg_method].rank(method="min", ascending=ascending).values


        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_name",
            "normalized_" + result_col,
            f"{metric}_rank_mean",
            result_col + "_min",
            result_col + "_max"
        ]

        if i == 0:
            tuned_agg_df = agg_tuned_alg_perf.copy()
        else:
            tuned_agg_df = tuned_agg_df.merge(agg_tuned_alg_perf[merge_cols], on=["alg_name", "dataset_name"])

    # save results

    # merge in alg type, for bookkeeping
    alg_type_df = analysis_df[["alg_name", "alg_type"]].drop_duplicates()
    tuned_agg_df = tuned_agg_df.merge(alg_type_df, on="alg_name", how="left")
    fold_tuned_df = fold_tuned_df.merge(alg_type_df, on="alg_name", how="left")

    if drop_default:
        agg_df_no_default = tuned_agg_df.copy()
        agg_df_no_default.to_csv("./cleaned_results/tuned_aggregated_results.csv")

        tuned_fold_df_no_default = fold_tuned_df.copy()
        tuned_fold_df_no_default.to_csv("./cleaned_results/tuned_fold_results.csv")
       
    else:
        agg_df_with_default = tuned_agg_df.copy()
        agg_df_with_default.to_csv("./cleaned_results/tuned_aggregated_results_with_default.csv")

        tuned_fold_df_with_default = fold_tuned_df.copy()
        tuned_fold_df_with_default.to_csv("./cleaned_results/tuned_fold_results_with_default.csv")


In [13]:
# take a peek
agg_df_no_default.head()

,alg_name,dataset_name,Accuracy__test_median,Accuracy__test_mean,time__train_median,time__train_mean,Accuracy__test_mean_min,Accuracy__test_mean_max,normalized_Accuracy__test_mean,Accuracy_rank_mean,normalized_F1__test_mean,F1_rank_mean,F1__test_mean_min,F1__test_mean_max,normalized_Log Loss__test_mean,Log Loss_rank_mean,Log Loss__test_mean_min,Log Loss__test_mean_max,alg_type
0,CatBoost,openml__Australian__146818,0.869565,0.872464,1.347650,1.393643,0.711594,0.872464,1.000000,1.0,1.000000,1.0,0.711594,0.872464,0.000000,1.0,0.302677,0.755920,gbdt
1,CatBoost,openml__Click_prediction_small__190408,0.839655,0.838565,14.031593,16.126213,0.831581,0.838565,1.000000,1.0,1.000000,1.0,0.831581,0.838565,0.000000,1.0,0.409856,5.816984,gbdt
2,CatBoost,openml__LED-display-domain-7digit__125921,0.720000,0.728000,0.696405,1.113755,0.698000,0.736000,0.789474,4.0,0.693412,5.0,0.697046,0.731609,0.000000,1.0,0.827409,2.539521,gbdt
3,CatBoost,openml__MiceProtein__146800,0.981481,0.980556,2.073627,3.307141,0.661111,0.998148,0.947802,7.0,0.954091,7.0,0.614618,0.998147,0.076744,9.0,0.011987,1.380437,gbdt
4,CatBoost,openml__PhishingWebsites__14952,0.961104,0.960561,5.313641,5.994810,0.927273,0.972953,0.728706,12.0,0.728706,12.0,0.927273,0.972953,0.148074,8.0,0.070005,0.192443,gbdt
